In [1]:
import pandas as pd

In [2]:
path = "../Data/Online_Data/trainning_data/corporate_attributes/"
outpath = "../Processed_Data/"

In [3]:
company_ar_assetsinfo = pd.read_csv(path+'company_ar_assetinfo.csv')
company_ar_assetsinfo.head()

<ipython-input-3-3293778e2e8a>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  company_ar_assetsinfo = pd.read_excel(path+'company_ar_assetinfo.xlsx')


,entid,ANCHEYEAR,ispublic,ASSGRO,LIAGRO,VENDINC,MAIBUSINC,PROGRO,NETINC,RATGRO,TOTEQU
0,932526009,2013,1,100.488892,-1.479566,0.283220,0.672107,-0.436201,0.976037,-0.080510,99.813856
1,47765794,2013,1,1985.294967,1986.898739,6.353740,8.011072,-0.265597,-1.095177,1.001812,0.791100
2,100233421,2014,1,-0.155075,-0.362538,-0.473252,-1.560821,-0.201790,-1.522116,1.182757,-0.811408
3,932526009,2014,1,99.997127,-0.438262,1.604003,-0.047880,2.271436,0.728556,-0.554784,99.437596
4,307832564,2014,1,0.096310,0.620053,-0.983819,0.628480,0.722161,0.233538,0.564325,0.866446


In [4]:
# only keep the latest data
# company_ar_assetsinfo_latest = company_ar_assetsinfo.query('ANCHEYEAR in [2016, 2017, 2018, 2019]')
company_ar_assetsinfo_latest = company_ar_assetsinfo.sort_values(by = 'ANCHEYEAR')

In [5]:
company_ar_assetsinfo_latest.shape

(4884, 11)

In [6]:
company_ar_assetsinfo_latest.columns

Index(['entid', 'ANCHEYEAR', 'ispublic', 'ASSGRO', 'LIAGRO', 'VENDINC',
       'MAIBUSINC', 'PROGRO', 'NETINC', 'RATGRO', 'TOTEQU'],
      dtype='object')

In [7]:
numerical_columns = ['ASSGRO', 'LIAGRO', 'VENDINC',
       'MAIBUSINC', 'PROGRO', 'NETINC', 'RATGRO', 'TOTEQU']

In [8]:
def get_numerical_df(df, columns):
    keep_columns = columns + ['entid']
    df_numerical = df[keep_columns].groupby("entid")
    df_numerical = df_numerical.mean()
    df_numerical = df_numerical.reset_index()
    return df_numerical

In [9]:
df_numerical = get_numerical_df(company_ar_assetsinfo_latest, numerical_columns)
df_numerical.shape

(2318, 9)

In [10]:
df_numerical.head()

,entid,ASSGRO,LIAGRO,VENDINC,MAIBUSINC,PROGRO,NETINC,RATGRO,TOTEQU
0,221197,NaN,NaN,NaN,NaN,NaN,NaN,0.389003,NaN
1,530656,572.628619,514.452836,180.805259,179.706849,-1.349599,-0.292298,10.467649,58.507187
2,951687,199.039445,-0.420831,-1.080154,0.096327,2.394148,-0.072355,0.326328,200.000452
3,1507679,2064.001746,2767.024702,3625.656199,3626.173157,-751.460106,-752.559701,-0.222469,-105.136521
4,2604506,NaN,NaN,-0.041023,NaN,NaN,-0.528690,0.003914,NaN


In [11]:
categorical_columns = ['ispublic']

In [12]:
def get_categorical_df(df, columns):
    keep_columns = columns + ['entid']
    df_categorical = df[keep_columns].drop_duplicates(subset = 'entid', keep = 'last')
    df_categorical = pd.get_dummies(df_categorical, columns = categorical_columns, drop_first=True)
    return df_categorical

In [13]:
df_categorical = get_categorical_df(company_ar_assetsinfo_latest, categorical_columns)

In [14]:
df_categorical.head()

,entid
846,191390596
857,1077483239
863,583034378
873,981015861
802,640967317


In [15]:
df_categorical.shape

(2318, 1)

In [16]:
df_new = pd.merge(df_numerical, df_categorical, on = "entid")
df_new.head()

,entid,ASSGRO,LIAGRO,VENDINC,MAIBUSINC,PROGRO,NETINC,RATGRO,TOTEQU
0,221197,NaN,NaN,NaN,NaN,NaN,NaN,0.389003,NaN
1,530656,572.628619,514.452836,180.805259,179.706849,-1.349599,-0.292298,10.467649,58.507187
2,951687,199.039445,-0.420831,-1.080154,0.096327,2.394148,-0.072355,0.326328,200.000452
3,1507679,2064.001746,2767.024702,3625.656199,3626.173157,-751.460106,-752.559701,-0.222469,-105.136521
4,2604506,NaN,NaN,-0.041023,NaN,NaN,-0.528690,0.003914,NaN


In [17]:
df_new.shape

(2318, 9)

In [18]:
de_columns = ['TOTEQU', 'ASSGRO', 'PROGRO', 'NETINC']

In [22]:
df_new[de_columns].isna().sum()

TOTEQU    271
ASSGRO    225
PROGRO    296
NETINC    170
dtype: int64

In [19]:
# for column in de_columns:
#     df_new[column] = df_new[column].replace(0, 0.01)

In [23]:
df_new['D_E_Ratio'] = df_new['LIAGRO'] / df_new['TOTEQU']
df_new['ROA'] = df_new['NETINC'] / df_new['ASSGRO']
df_new['ROE'] = df_new['LIAGRO'] / df_new['TOTEQU']
df_new['Debit_Ration'] = df_new['LIAGRO'] / df_new['ASSGRO']
df_new['SH_Equity_Ratio'] = df_new['TOTEQU'] / df_new['ASSGRO']
df_new['Debit_Ratio'] = df_new['LIAGRO'] / df_new['ASSGRO']
df_new['Efficiency_Ratio'] = df_new['NETINC'] / df_new['PROGRO']
df_new['Effective_Tax_Rate'] = df_new['RATGRO'] / df_new['NETINC']

In [25]:
df_new['D_E_Ratio'].isna().sum()

295

In [26]:
df_new.to_csv(outpath+'company_ar_assetsinfo_processed.csv', index = 0)